# ***Detección de URLs de Phishing mediante un Enfoque Híbrido con Machine Learning***

El presente trabajo se realiza con el objetivo de desarrollar un modelo híbrido para la detección de URLs de phishing, utilizando un enfoque basado en técnicas de aprendizaje automático. Para ello, se propone un modelo de ***stacking*** que combina múltiples algoritmos de clasificación, optimizando su desempeño a través de un metamodelo basado en ***Regresión Logística***.

El objetivo principal es evaluar el desempeño de diferentes modelos individuales y comparar su efectividad frente a la estrategia híbrida. Dentro de los modelos de ***benchmark*** utilizados en el estudio se encuentran:

- ***K-Nearest Neighbors (KNN)***
- ***Regresión Ridge y Lasso***
- ***Clasificador Bayesiano***
- ***Árboles de Decisión (DT)***
- ***Random Forest (RF)***
- ***XGBoost***
- ***Máquinas de Soporte Vectorial (SVM)***
- ***Perceptrón Multicapa (MLP)***

El análisis se realizará sobre el conjunto de datos ***[PhiUSIIL Phishing URL](https://archive.ics.uci.edu/dataset/967/phiusiil+phishing+url+dataset)***, que contiene características relevantes extraídas de URLs legítimas y de phishing. A través de este estudio, se espera demostrar que la combinación de múltiples modelos a través de ***stacking*** puede mejorar la capacidad predictiva y generalización en la detección de sitios maliciosos.

El resto del trabajo se encuentra organizado de la siguiente manera:

- ***Información del Conjunto de Datos***
- ***Análisis Exploratorio de Datos***
- ***Fundamentación Matemática de los Modelos***
- ***Implementación de los Modelos***
- ***Benchmark***
- ***Conclusiones***
- ***Referencias***


## ***Información del Conjunto de datos***

Primero, cargaremos el dataset desde el repositorio de ***[UCI](https://archive.ics.uci.edu/)*** mediante la siguiente línea de código.

In [12]:
from ucimlrepo import fetch_ucirepo 

# fetch dataset 
phiusiil_phishing_url_website = fetch_ucirepo(id=967) 

Con esto, obtenemos un objeto de tipo `ucimlrepo.dotdict.dotdict`, el cual asignaremos a nuestros conjuntos `X` e `y` de la siguiente manera:

In [13]:
# data (as pandas dataframes) 
X = phiusiil_phishing_url_website.data.features 
y = phiusiil_phishing_url_website.data.targets 

Podemos acceder a los metadatos del dataset, donde encontraremos información útil como el **ID del dataset (967)**, el resumen (*abstract*), las tareas de investigación (*tasks*), y otros detalles importantes. Este dataset, denominado ***PhiUSIIL Phishing URL (Website)***, contiene un total de **235,795 instancias** y **54 características**, combinando datos **reales, categóricos e enteros**. Su propósito principal es la **clasificación de URLs en legítimas o phishing**, con características extraídas tanto del código fuente de las páginas web como de la estructura de la URL. Fue creado en **2024** y se encuentra disponible en el repositorio de ***[UCI](https://archive.ics.uci.edu/dataset/967/phiusiil+phishing+url+dataset)***.

### ***Metadatos***

In [14]:
# metadata
display(phiusiil_phishing_url_website.metadata) 

{'uci_id': 967,
 'name': 'PhiUSIIL Phishing URL (Website)',
 'repository_url': 'https://archive.ics.uci.edu/dataset/967/phiusiil+phishing+url+dataset',
 'data_url': 'https://archive.ics.uci.edu/static/public/967/data.csv',
 'abstract': 'PhiUSIIL Phishing URL Dataset is a substantial dataset comprising 134,850 legitimate and 100,945 phishing URLs. Most of the URLs we analyzed, while constructing the dataset, are the latest URLs. Features are extracted from the source code of the webpage and URL. Features such as CharContinuationRate, URLTitleMatchScore, URLCharProb, and TLDLegitimateProb are derived from existing features.',
 'area': 'Computer Science',
 'tasks': ['Classification'],
 'characteristics': ['Tabular'],
 'num_instances': 235795,
 'num_features': 54,
 'feature_types': ['Real', 'Categorical', 'Integer'],
 'demographics': [],
 'target_col': ['label'],
 'index_col': None,
 'has_missing_values': 'no',
 'missing_values_symbol': None,
 'year_of_dataset_creation': 2024,
 'last_updat

### ***Variables***

#### **Información de la URL y el Dominio**  
- **`URL`**: Dirección completa de la página web analizada.  
- **`URLLength`**: Longitud total de la URL en número de caracteres.  
- **`Domain`**: Dominio principal extraído de la URL.  
- **`DomainLength`**: Longitud del dominio en caracteres.  
- **`IsDomainIP`**: Indica si el dominio es una dirección IP en lugar de un nombre de dominio ($1$ si es una IP, $0$ si no).  
- **`TLD`**: Extensión del dominio de nivel superior (ejemplo: `.com`, `.org`).  
- **`TLDLength`**: Longitud del TLD en caracteres.  
- **`NoOfSubDomain`**: Número de subdominios presentes en la URL.  

#### **Características basadas en similitud y probabilidad**  
- **`URLSimilarityIndex`**: Índice de similitud de la URL con otras URLs de phishing conocidas.  
- **`TLDLegitimateProb`**: Probabilidad de que el TLD pertenezca a un sitio legítimo.  
- **`URLCharProb`**: Probabilidad de que la composición de caracteres en la URL corresponda a una página legítima.  
- **`DomainTitleMatchScore`**: Puntaje que mide la similitud entre el dominio y el título de la página.  
- **`URLTitleMatchScore`**: Puntaje que mide la similitud entre la URL y el título de la página.  

#### **Análisis de caracteres en la URL**  
- **`CharContinuationRate`**: Tasa de continuidad de caracteres en la URL, mide patrones de escritura sospechosos.  
- **`NoOfLettersInURL`**: Cantidad total de letras en la URL.  
- **`LetterRatioInURL`**: Proporción de letras respecto al total de caracteres en la URL.  
- **`NoOfDegitsInURL`**: Número total de dígitos en la URL.  
- **`DegitRatioInURL`**: Proporción de dígitos respecto al total de caracteres en la URL.  
- **`NoOfEqualsInURL`**: Número de signos `=` en la URL.  
- **`NoOfQMarkInURL`**: Número de signos de interrogación `?` en la URL.  
- **`NoOfAmpersandInURL`**: Número de símbolos `&` en la URL.  
- **`NoOfOtherSpecialCharsInURL`**: Número total de otros caracteres especiales en la URL.  
- **`SpacialCharRatioInURL`**: Proporción de caracteres especiales en la URL.  

#### **Presencia de técnicas de ofuscación**  
- **`HasObfuscation`**: Indica si la URL tiene técnicas de ofuscación ($1$ si tiene, $0$ si no).  
- **`NoOfObfuscatedChar`**: Número de caracteres que forman parte de una técnica de ofuscación.  
- **`ObfuscationRatio`**: Proporción de caracteres de ofuscación respecto al total de caracteres de la URL.  

#### **Protocolo y seguridad**  
- **`IsHTTPS`**: Indica si la URL usa el protocolo seguro HTTPS ($1$ si sí, $0$ si usa HTTP).  
- **`Robots`**: Indica si la página tiene un archivo `robots.txt` ($1$ si sí, $0$ si no).  
- **`IsResponsive`**: Indica si el sitio web responde correctamente a las solicitudes del usuario ($1$ si sí, $0$ si no).  

#### **Estructura del código fuente**  
- **`LineOfCode`**: Número total de líneas en el código fuente de la página.  
- **`LargestLineLength`**: Longitud de la línea más larga en el código fuente.  
- **`HasTitle`**: Indica si la página tiene una etiqueta `<title>` definida ($1$ si sí, $0$ si no).  
- **`Title`**: Texto contenido en la etiqueta `<title>`.  

#### **Redirecciones y formularios**  
- **`NoOfURLRedirect`**: Número total de redirecciones de la URL a otras páginas.  
- **`NoOfSelfRedirect`**: Número de redirecciones dentro del mismo dominio.  
- **`HasExternalFormSubmit`**: Indica si hay formularios en la página que envían datos a un dominio externo ($1$ si sí, $0$ si no).  
- **`HasSubmitButton`**: Indica si la página tiene botones de envío de formularios.  
- **`HasHiddenFields`**: Indica si la página tiene campos de formulario ocultos.  
- **`HasPasswordField`**: Indica si la página tiene un campo de contraseña.  

#### **Elementos gráficos y multimedia**  
- **`NoOfPopup`**: Número de ventanas emergentes (pop-ups) detectadas.  
- **`NoOfiFrame`**: Número de `iframe` incrustados en la página.  
- **`HasFavicon`**: Indica si la página tiene un favicon ($1$ si sí, $0$ si no).  
- **`NoOfImage`**: Número total de imágenes en la página.  
- **`NoOfCSS`**: Número total de archivos CSS en la página.  
- **`NoOfJS`**: Número total de archivos JavaScript en la página.  

#### **Referencias y enlaces**  
- **`NoOfSelfRef`**: Número de enlaces dentro del mismo dominio.  
- **`NoOfEmptyRef`**: Número de enlaces vacíos (`href="#"`).  
- **`NoOfExternalRef`**: Número de enlaces externos a otros dominios.  

#### **Presencia de elementos financieros y sociales**  
- **`Bank`**: Indica si la página contiene palabras clave relacionadas con bancos.  
- **`Pay`**: Indica si la página contiene términos relacionados con pagos.  
- **`Crypto`**: Indica si la página menciona términos relacionados con criptomonedas.  
- **`HasSocialNet`**: Indica si la página contiene enlaces a redes sociales.  

#### **Información de derechos y seguridad**  
- **`HasCopyrightInfo`**: Indica si la página menciona información de derechos de autor.  

#### **Variable objetivo**  
- **`label`**: Variable de clasificación binaria que indica si la URL es phishing ($0$) o legítima ($1$).  

## ***Análisis Exploratorio de Datos***

En esta sección, se presenta un ***Análisis Exploratorio de Datos (EDA)*** sobre el dataset empleado, con el objetivo de identificar el comportamiento de las variables, detectar valores faltantes, así como analizar patrones relevantes antes de proceder con la construcción de los modelos de clasificación.

### ***Estadísticas Descriptivas***

In [28]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Enable better formatting in Jupyter Notebook
pd.set_option('display.width', 1000)  # Adjust output width
pd.set_option('display.colheader_justify', 'center')  # Align column headers

# Improve DataFrame styling (works in Jupyter)
def style_dataframe(df):
    return df.style.set_properties(**{
        'background-color': '#f4f4f4',
        'border-color': 'black',
        'color': 'black',
        'font-size': '12px',
    }).set_table_styles([{
        'selector': 'thead th',
        'props': [('background-color', '#40466e'), ('color', 'white')]
    }])

In [23]:
# libraries
import numpy as np

Podemos obtener una previsualización del dataset utilizando la función ***`.head()`***, que muestra las primeras filas del conjunto de datos. Esto permite inspeccionar su estructura, tipos de datos y valores iniciales antes de profundizar en el análisis exploratorio.

In [29]:
df = pd.concat([X, y], axis=1)
df.head()

,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,NoOfObfuscatedChar,ObfuscationRatio,NoOfLettersInURL,LetterRatioInURL,NoOfDegitsInURL,DegitRatioInURL,NoOfEqualsInURL,NoOfQMarkInURL,NoOfAmpersandInURL,NoOfOtherSpecialCharsInURL,SpacialCharRatioInURL,IsHTTPS,LineOfCode,LargestLineLength,HasTitle,Title,DomainTitleMatchScore,URLTitleMatchScore,HasFavicon,Robots,IsResponsive,NoOfURLRedirect,NoOfSelfRedirect,HasDescription,NoOfPopup,NoOfiFrame,HasExternalFormSubmit,HasSocialNet,HasSubmitButton,HasHiddenFields,HasPasswordField,Bank,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,https://www.southbankmosaics.com,31,www.southbankmosaics.com,24,0,com,100.00,1.00,0.52,0.06,3,1,0,0,0.00,18,0.58,0,0.00,0,0,0,1,0.03,1,558,9381,1,à¸‚à¹ˆà¸²à¸§à¸ªà¸” à¸‚à¹ˆà¸²à¸§à¸§à¸±à¸™à¸™à¸µ...,0.00,0.00,0,1,1,0,0,0,0,1,0,0,1,1,0,1,0,0,1,34,20,28,119,0,124,1
1,https://www.uni-mainz.de,23,www.uni-mainz.de,16,0,de,100.00,0.67,0.03,0.05,2,1,0,0,0.00,9,0.39,0,0.00,0,0,0,2,0.09,1,618,9381,1,johannes gutenberg-universitÃ¤t mainz,55.56,55.56,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,50,9,8,39,0,217,1
2,https://www.voicefmradio.co.uk,29,www.voicefmradio.co.uk,22,0,uk,100.00,0.87,0.03,0.06,2,2,0,0,0.00,15,0.52,0,0.00,0,0,0,2,0.07,1,467,682,1,voice fm southampton,46.67,46.67,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,10,2,7,42,2,5,1
3,https://www.sfnmjournal.com,26,www.sfnmjournal.com,19,0,com,100.00,1.00,0.52,0.06,3,1,0,0,0.00,13,0.50,0,0.00,0,0,0,1,0.04,1,6356,26824,1,home page: seminars in fetal and neonatal medi...,0.00,0.00,0,1,1,0,0,0,1,12,0,1,1,1,0,0,1,1,1,3,27,15,22,1,31,1
4,https://www.rewildingargentina.org,33,www.rewildingargentina.org,26,0,org,100.00,1.00,0.08,0.06,3,1,0,0,0.00,20,0.61,0,0.00,0,0,0,1,0.03,1,6089,28404,1,fundaciÃ³n rewilding argentina,100.00,100.00,0,1,1,1,1,1,0,2,0,1,1,1,0,1,1,0,1,244,15,34,72,1,85,1


Se observa una mezcla de datos tanto **numéricos** como **categóricos**, lo que sugiere la necesidad de un tratamiento adecuado para cada tipo de variable. Además, se evidencia que el problema es de **clasificación binaria**, donde el valor $0$ representa una URL clasificada como **phishing**, mientras que el valor $1$ indica una **URL legítima**.

In [30]:
df.shape

(235795, 55)

Se evidencia un dataset con $235,795$ registros, 54 variables predictoras y una variable objetivo.

In [31]:
df.describe()

,URLLength,DomainLength,IsDomainIP,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,NoOfSubDomain,HasObfuscation,NoOfObfuscatedChar,ObfuscationRatio,NoOfLettersInURL,LetterRatioInURL,NoOfDegitsInURL,DegitRatioInURL,NoOfEqualsInURL,NoOfQMarkInURL,NoOfAmpersandInURL,NoOfOtherSpecialCharsInURL,SpacialCharRatioInURL,IsHTTPS,LineOfCode,LargestLineLength,HasTitle,DomainTitleMatchScore,URLTitleMatchScore,HasFavicon,Robots,IsResponsive,NoOfURLRedirect,NoOfSelfRedirect,HasDescription,NoOfPopup,NoOfiFrame,HasExternalFormSubmit,HasSocialNet,HasSubmitButton,HasHiddenFields,HasPasswordField,Bank,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
count,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00,235795.00
mean,34.57,21.47,0.00,78.43,0.85,0.26,0.06,2.76,1.16,0.00,0.02,0.00,19.43,0.52,1.88,0.03,0.06,0.03,0.03,2.34,0.06,0.78,1141.90,12789.53,0.86,50.13,52.12,0.36,0.27,0.62,0.13,0.04,0.44,0.22,1.59,0.04,0.46,0.41,0.38,0.10,0.13,0.24,0.02,0.49,26.08,6.33,10.52,65.07,2.38,49.26,0.57
std,41.31,9.15,0.05,28.98,0.22,0.25,0.01,0.60,0.60,0.05,1.88,0.00,29.09,0.12,11.89,0.07,0.93,0.19,0.84,3.53,0.03,0.41,3419.95,152201.10,0.35,49.68,49.60,0.48,0.44,0.48,0.34,0.20,0.50,3.87,5.76,0.21,0.50,0.49,0.48,0.30,0.33,0.43,0.15,0.50,79.41,74.87,22.31,176.69,17.64,161.03,0.49
min,13.00,4.00,0.00,0.16,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,22.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,23.00,16.00,0.00,57.02,0.68,0.01,0.05,2.00,1.00,0.00,0.00,0.00,10.00,0.43,0.00,0.00,0.00,0.00,0.00,1.00,0.04,1.00,18.00,200.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
50%,27.00,20.00,0.00,100.00,1.00,0.08,0.06,3.00,1.00,0.00,0.00,0.00,14.00,0.52,0.00,0.00,0.00,0.00,0.00,1.00,0.05,1.00,429.00,1090.00,1.00,75.00,100.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.00,2.00,6.00,12.00,0.00,10.00,1.00
75%,34.00,24.00,0.00,100.00,1.00,0.52,0.06,3.00,1.00,0.00,0.00,0.00,20.00,0.59,0.00,0.00,0.00,0.00,0.00,3.00,0.08,1.00,1277.00,8047.00,1.00,100.00,100.00,1.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,29.00,8.00,15.00,88.00,1.00,57.00,1.00
max,6097.00,110.00,1.00,100.00,1.00,0.52,0.09,13.00,10.00,1.00,447.00,0.35,5191.00,0.93,2011.00,0.68,176.00,4.00,149.00,499.00,0.40,1.00,442666.00,13975732.00,1.00,100.00,100.00,1.00,1.00,1.00,1.00,1.00,1.00,602.00,1602.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,8956.00,35820.00,6957.00,27397.00,4887.00,27516.00,1.00


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235795 entries, 0 to 235794
Data columns (total 55 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   URL                         235795 non-null  object 
 1   URLLength                   235795 non-null  int64  
 2   Domain                      235795 non-null  object 
 3   DomainLength                235795 non-null  int64  
 4   IsDomainIP                  235795 non-null  int64  
 5   TLD                         235795 non-null  object 
 6   URLSimilarityIndex          235795 non-null  float64
 7   CharContinuationRate        235795 non-null  float64
 8   TLDLegitimateProb           235795 non-null  float64
 9   URLCharProb                 235795 non-null  float64
 10  TLDLength                   235795 non-null  int64  
 11  NoOfSubDomain               235795 non-null  int64  
 12  HasObfuscation              235795 non-null  int64  
 13  NoOfObfuscated

In [33]:
df.isnull().sum()

URL                           0
URLLength                     0
Domain                        0
DomainLength                  0
IsDomainIP                    0
TLD                           0
URLSimilarityIndex            0
CharContinuationRate          0
TLDLegitimateProb             0
URLCharProb                   0
TLDLength                     0
NoOfSubDomain                 0
HasObfuscation                0
NoOfObfuscatedChar            0
ObfuscationRatio              0
NoOfLettersInURL              0
LetterRatioInURL              0
NoOfDegitsInURL               0
DegitRatioInURL               0
NoOfEqualsInURL               0
NoOfQMarkInURL                0
NoOfAmpersandInURL            0
NoOfOtherSpecialCharsInURL    0
SpacialCharRatioInURL         0
IsHTTPS                       0
LineOfCode                    0
LargestLineLength             0
HasTitle                      0
Title                         0
DomainTitleMatchScore         0
URLTitleMatchScore            0
HasFavic

In [5]:
y.value_counts()

label
1        134850
0        100945
Name: count, dtype: int64